<a href="https://colab.research.google.com/github/gloomisbest/Machine-Learning/blob/master/keras_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 113598 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [2]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
%cd drive//Colab Notebooks/B&P_test
!ls

/content/drive/Colab Notebooks/B&P_test
model.png  test  train


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.optimizers import Adam

import tensorflow as tf

tf_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
K.set_session(tf.Session(config=tf_config))

Using TensorFlow backend.


In [0]:
# dimensions of our images.
img_width, img_height = 128, 128

train_data_dir = './train'
validation_data_dir = './test'
nb_train_samples = 500
nb_validation_samples = 60
epochs = 20
batch_size = 64

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [6]:
# Another way to build your CNN
model = Sequential()

# Conv layer 1 output shape (64, 128, 128)
model.add(Convolution2D(
    batch_input_shape=(None, 128, 128, 3),
    filters=32,
    kernel_size=5,
    strides=1,
    padding='same',     # Padding method
    data_format='channels_first',
))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Pooling layer 1 (max pooling) output shape (32, 48, 48)
model.add(MaxPooling2D(
    pool_size=2,
    strides=2,
    padding='same',    # Padding method
    data_format='channels_first',
))


# Conv layer 2 output shape (64, 48, 48)
model.add(Convolution2D(32, 5, strides=1, padding='same', data_format='channels_first'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Pooling layer 2 (max pooling) output shape (64, 24, 24)
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))

# Conv layer 2 output shape (64, 48, 48)
model.add(Convolution2D(64, 5, strides=1, padding='same', data_format='channels_first'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Pooling layer 2 (max pooling) output shape (64, 24, 24)
model.add(MaxPooling2D(2, 2, 'same', data_format='channels_first'))


# Fully connected layer 1 input shape (64 * 7 * 7) = (3136), output shape (1024)
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# Fully connected layer 2 to shape (10) for 10 classes
model.add(Dense(1))
model.add(Activation('sigmoid'))

adam = Adam(lr=1e-4)

# We add metrics to get more results you want to see
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

#model.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    classes=['Bulbasaur', 'Pikachu'],
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    classes=['Bulbasaur', 'Pikachu'],
    class_mode='binary')


Found 500 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [10]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint
import keras


history = History()
model_checkpoint = ModelCheckpoint('temp_model.hdf5', monitor='loss', save_best_only=True)
tb_cb = keras.callbacks.TensorBoard(log_dir='log', write_images=1, histogram_freq=0)
# 设置log的存储位置，将网络权值以图片格式保持在tensorboard中显示，设置每一个周期计算一次网络的
# 权值，每层输出值的分布直方图
callbacks = [
        history,
        model_checkpoint,
        tb_cb]



history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples ,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples)

Epoch 1/20
500/500 [==============================] - 220s 440ms/step - loss: 0.0530 - acc: 0.9803 - val_loss: 0.1911 - val_acc: 0.8500
Epoch 2/20
500/500 [==============================] - 215s 429ms/step - loss: 0.0498 - acc: 0.9820 - val_loss: 0.1999 - val_acc: 0.8833
Epoch 3/20
500/500 [==============================] - 214s 428ms/step - loss: 0.0493 - acc: 0.9820 - val_loss: 0.1947 - val_acc: 0.8833
Epoch 4/20
500/500 [==============================] - 215s 430ms/step - loss: 0.0471 - acc: 0.9823 - val_loss: 0.1617 - val_acc: 0.9333
Epoch 5/20
500/500 [==============================] - 214s 428ms/step - loss: 0.0462 - acc: 0.9833 - val_loss: 0.1911 - val_acc: 0.8833
Epoch 6/20
500/500 [==============================] - 215s 430ms/step - loss: 0.0443 - acc: 0.9833 - val_loss: 0.1685 - val_acc: 0.9000
Epoch 7/20
500/500 [==============================] - 212s 425ms/step - loss: 0.0435 - acc: 0.9836 - val_loss: 0.1896 - val_acc: 0.8833
Epoch 8/20
500/500 [============================

In [0]:
from matplotlib import pyplot as plt
history=history
plt.plot()
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [12]:

#保存模型和权重
model.save('fine_tune_model.h5')
model.save_weights('fine_tune_model_weight')
print(history.history)

{'val_loss': [0.1911401445666949, 0.1999170608818531, 0.19468707342942557, 0.16170232196648915, 0.19114599600434304, 0.16850365847349166, 0.18956777627269428, 0.2174483557542165, 0.1482274127503236, 0.19159504945079486, 0.2447379656136036, 0.17454172670841217, 0.18275699019432068, 0.1597178506354491, 0.1527642399072647, 0.16872335722049078, 0.1850239172577858, 0.15770002404848735, 0.11343845936159293, 0.1676344874004523], 'val_acc': [0.8500000238418579, 0.8833333253860474, 0.8833333253860474, 0.9333333373069763, 0.8833333253860474, 0.8999999761581421, 0.8833333253860474, 0.8833333253860474, 0.9333333373069763, 0.8999999761581421, 0.8999999761581421, 0.9333333373069763, 0.9166666865348816, 0.9333333373069763, 0.9166666865348816, 0.9333333373069763, 0.8999999761581421, 0.9166666865348816, 0.949999988079071, 0.8666666746139526], 'loss': [0.05311738620975823, 0.04964903721811457, 0.04944874834241157, 0.047032129858953005, 0.046034694768882674, 0.044340415051096774, 0.04320834599621737, 0.0

In [33]:
import numpy as np
from keras.models import load_model
import os
from PIL import Image

model = load_model('fine_tune_model.h5')  #选取自己的.h模型名称
img_path = r'./test_colab'

for i in os.listdir(img_path):
  #需要用reshape定义出例子的个数，图片的 通道数，图片的长与宽。具体的参加keras文档
  img = Image.open(os.path.join(img_path,i))
  img_show = img
  img = np.array(img)
  img = img.reshape(1,128, 128, 3)/255.
  predict = model.predict_classes(img)
  print (i,'识别为：')
  if predict == '[[1]]':
    print('Bulbasaur')
  else:
    print('Pikachu')




test004.jpg 识别为：
Pikachu
test008.jpg 识别为：
Pikachu
test003.jpg 识别为：
Pikachu
test006.jpg 识别为：
Pikachu
test009.jpg 识别为：
Pikachu
test005.jpg 识别为：
Pikachu
test007.jpg 识别为：
Pikachu
test001.jpg 识别为：
Pikachu
test002.jpg 识别为：
Pikachu
test010.jpg 识别为：
Pikachu


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
